In [45]:
import pandas as pd
import numpy as np
from scipy.stats import uniform
from scipy.stats import expon
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
import dill
import pickle
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
%load_ext autoreload
%autoreload 2
import classification_module as classmod
import features_extraction_funcs as feex
import feats_extr_module as feex_module
%aimport

pipe_names = [#'V0001_PCA_80','V0001_PCA_70','V0001_PCA_5components',
              #'V001_PCA_80',
              'V001_PCA_70',
              'V001_PCA_5components',
              #'V01_PCA_90',
              'V01_PCA_85',
              'V01_PCA_80',
              'V01_PCA_5components',
              #'V09_PCA_95',
              'V09_PCA_90',
              'V09_PCA_80',
              'V09_PCA_5components'
            ]
pipes = [
    #make_pipeline(VarianceThreshold(0.001), StandardScaler(),
    #              PCA(n_components = 0.80, whiten = True, svd_solver = 'full')),
    #make_pipeline(VarianceThreshold(0.001), StandardScaler(),
    #              PCA(n_components = 0.70, whiten = True, svd_solver = 'full')),
    #make_pipeline(VarianceThreshold(0.001), StandardScaler(),
    #              PCA(n_components = 5, whiten = True, svd_solver = 'full')),
    
    #make_pipeline(VarianceThreshold(0.01), StandardScaler(),
    #              PCA(n_components = 0.80, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.01), StandardScaler(),
                  PCA(n_components = 0.70, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.01), StandardScaler(),
                  PCA(n_components = 5, whiten = True, svd_solver = 'full')),
    
    #make_pipeline(VarianceThreshold(0.1), StandardScaler(),
    #              PCA(n_components = 0.90, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.1), StandardScaler(),
                  PCA(n_components = 0.85, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.1), StandardScaler(),
                  PCA(n_components = 0.80, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.1), StandardScaler(),
                  PCA(n_components = 5, whiten = True, svd_solver = 'full')),
    
    #make_pipeline(VarianceThreshold(0.9), StandardScaler(),
    #              PCA(n_components = 0.95, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.9), StandardScaler(),
                  PCA(n_components = 0.90, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.9), StandardScaler(),
                  PCA(n_components = 0.80, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.9), StandardScaler(),
                  PCA(n_components = 5, whiten = True, svd_solver = 'full')),
       ]


#pipe_names = ['V09_PCA_5components']
#pipes = [make_pipeline(VarianceThreshold(0.9), StandardScaler(),
#                  PCA(n_components = 5, whiten = True, svd_solver = 'full'))]
model_names_v1 = ['LogisticRegression',
                  'RandomForestClassifier', 'DecisionTreeClassifier', 'KNeighborsClassifier',
                    'SVC', 'MLPClassifier']

model_names_v1 = ['LogisticRegression',
                  'RandomForestClassifier', 'DecisionTreeClassifier', 'KNeighborsClassifier',
                  'MLPClassifier']

models_v1 = [LogisticRegression(),
             RandomForestClassifier(), DecisionTreeClassifier(), KNeighborsClassifier(), SVC(probability=True),
               MLPClassifier()]

models_v1 = [LogisticRegression(),
             RandomForestClassifier(), DecisionTreeClassifier(), KNeighborsClassifier(),
             MLPClassifier()]
##{'C': expon(scale=0.2), 'degree': [2,3,4,5,6,7,8], 'kernel': ['rbf', 'poly', 'linear', 'sigmoid'],
##    'coef0': [0.0, 0.5, 0.75, 1.0], 'shrinking': [True, False], 'tol': np.arange(1e-5,0.7,0.00001)},
models_params_v1 = [
    {'penalty': ['l2'], 'tol': expon(scale=0.2), 'C': expon(scale=0.2),
     'class_weight': ['balanced', None], 'solver': ['newton-cg', 'saga', 'liblinear']},
    {'n_estimators': [100,150,200,250,300,350], 'max_depth': np.arange(5,105,5),
     'min_samples_leaf': [10, 50, 150, 200, 250, 125, 75], 'max_features': ['sqrt', 'log2']},
    {'criterion': ['gini', 'entropy'], 'max_depth': [None, 20],
     'min_samples_split': np.arange(0.01,1,0.01),
    'max_features': ['sqrt', 'log2', 0.5], 'class_weight': ['balanced', None]},
    {'n_neighbors': [3,4,5,6,7,8], 'weights': ['uniform', 'distance'],
     'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']},
    {'hidden_layer_sizes': [(100,),(100,100,),(100,100,50,),(200,50,),(200,50,50,),(200,50,50,50)],
     'solver': ['adam','sgd','lbfgs'], 'activation': ['relu', 'identity', 'tanh', 'logistic'],
    'alpha': np.arange(0.0001,0.05,0.0001)}
]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Modules to reload:
all-except-skipped

Modules to skip:



In [2]:
dill.load_session('features_extracted_dataset_splitted.db')

In [3]:
mlp_bench = classmod.ModelsBenchmark(model_names_v1, models_v1, models_params_v1)
mlp_bench.fitTransformators(names=pipe_names,
                             transformators=pipes,
                             X=X_train)

In [4]:
y_labels = np.unique(y_train)
score_dict = {'Accuracy': make_scorer(accuracy_score), 'F1': make_scorer(f1_score, average='weighted'),
             'Log_Loss': make_scorer(log_loss,normalize=True,greater_is_better=False, needs_proba=True)} 
mlp_bench.runBenchmarkOnLoaded(X_train, y_train,  prio_score='Log_Loss', folds=3, iters=30, randomized=True,
                                    verbosity=3, print_shapes=True,
                               scoring=score_dict)
rootDir = '''D:/PROGRAMMING_PROJS/FoodMachineLearning/FoodMLrepo/FoodML/foodml_src/'''
mlp_bench.saveToPickles(rootDir,'randomized_results_much_more')

Before PCA, x: 8322; y: 16680
After PCA, x: 8322; y: 89
Finding best parameters for LogisticRegression_V001_PCA_70:
Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.3min finished


Evaluating LogisticRegression_V001_PCA_70...

Finding best parameters for RandomForestClassifier_V001_PCA_70:
Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  3.2min finished


Evaluating RandomForestClassifier_V001_PCA_70...

Finding best parameters for DecisionTreeClassifier_V001_PCA_70:
Fitting 3 folds for each of 30 candidates, totalling 90 fits


...

Finding best parameters for KNeighborsClassifier_V09_PCA_5components:
Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:   35.8s finished


Evaluating KNeighborsClassifier_V09_PCA_5components...

Finding best parameters for MLPClassifier_V09_PCA_5components:
Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  4.0min finished
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Evaluating MLPClassifier_V09_PCA_5components...



In [5]:
mlp_bench.models_info_dict

C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_Accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_Accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_Accuracy'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C

{'LogisticRegression_V001_PCA_70': ({'mean_fit_time': array([2.53256464, 6.14291334, 2.39559706, 0.74168444, 0.76761548,
          1.72040153, 1.38529563, 4.39126269, 3.04053919, 5.33341074,
          2.7742509 , 5.43680064, 1.09208067, 2.67086053, 5.24697502,
          1.39360833, 3.95576048, 3.68182453, 4.84637904, 5.54418127,
          3.63062859, 1.43217142, 5.56379366, 3.94711614, 2.48103436,
          1.91853809, 5.92848587, 0.81980824, 1.07146883, 2.70676525]),
   'std_fit_time': array([0.20410313, 0.12796058, 0.24522868, 0.00896976, 0.03410913,
          0.03002244, 0.12139547, 0.25678791, 0.04219096, 0.00490791,
          0.02497147, 0.46500178, 0.01835442, 0.20219266, 0.13593811,
          0.02896666, 0.0372494 , 0.07864088, 0.28623913, 0.03518537,
          0.25463021, 0.12926866, 0.1769907 , 0.16596978, 0.2567767 ,
          0.03184426, 0.16601789, 0.03403636, 0.01861032, 0.07277668]),
   'mean_score_time': array([0.05485304, 0.04022622, 0.06183441, 0.0322473 , 0.04255239,


In [6]:
mlp_bench.best_models_dict

{'LogisticRegression_V001_PCA_70': LogisticRegression(C=0.4546229691182355, class_weight=None, dual=False,
           fit_intercept=True, intercept_scaling=1, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           solver='newton-cg', tol=0.22473909055263924, verbose=0,
           warm_start=False),
 'RandomForestClassifier_V001_PCA_70': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=35, max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=10, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=350, n_jobs=1,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'DecisionTreeClassifier_V001_PCA_70': DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
             max_depth=None, max_features=0.5, max_leaf_nodes=None,
             m

In [7]:
mlp_bench.best_models_results

{'LogisticRegression_V001_PCA_70': (0.7772170151405912,
  0.7742983725062028,
  0.8608661957702183,
  array([[285,   3,   0, ...,   2,  21,   2],
         [  0, 172,   0, ...,   6,   0,   0],
         [  0,   0, 197, ...,   0,   0,   0],
         ...,
         [  0,   0,   0, ..., 197,   0,   0],
         [ 11,   0,   0, ...,   0, 188,   2],
         [  4,   0,   2, ...,   0,   0, 166]], dtype=int64),
  '             precision    recall  f1-score   support\n\n      APPLE       0.65      0.61      0.63       466\n    APRICOT       0.79      0.85      0.82       202\n  AUBERGINE       0.95      0.98      0.97       200\n    AVOCADO       0.92      0.97      0.95       209\n     BANANA       0.59      0.45      0.51       213\n       BEAN       0.92      0.94      0.93       195\n      BREAD       0.81      0.85      0.82       220\n        BUN       0.87      0.84      0.86       205\n     CARROT       0.80      0.67      0.73       199\n     CHEESE       0.99      0.99      0.99       1

In [41]:
best_results_filename = rootDir + "_best_results_" + '_randomized_results_much_more' + '.sav'
pickle.dump(mlp_bench.best_models_results, open(best_results_filename,'wb'))

In [8]:
mlp_bench.best_models_dict['KNeighborsClassifier_V09_PCA_5components'].fit(
    mlp_bench.transformators['V09_PCA_5components'].transform(X_train),y_train)

KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=8, p=2,
           weights='distance')

In [9]:
mlp_bench.best_models_dict.items()

dict_items([('LogisticRegression_V001_PCA_70', LogisticRegression(C=0.4546229691182355, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='newton-cg', tol=0.22473909055263924, verbose=0,
          warm_start=False)), ('RandomForestClassifier_V001_PCA_70', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=35, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=350, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)), ('DecisionTreeClassifier_V001_PCA_70', DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=None, max_features=0.5, max_leaf_nodes=None,
           

In [10]:
mlp_bench.evaluate_model(mlp_bench.transformators['V09_PCA_5components'].transform(X_train),y_train,
                         np.unique(y_train),mlp_bench.best_models_dict['KNeighborsClassifier_V09_PCA_5components'])

(1.0, 1.0, 3.772517025997449e-14, array([[466,   0,   0, ...,   0,   0,   0],
        [  0, 202,   0, ...,   0,   0,   0],
        [  0,   0, 200, ...,   0,   0,   0],
        ...,
        [  0,   0,   0, ..., 208,   0,   0],
        [  0,   0,   0, ...,   0, 266,   0],
        [  0,   0,   0, ...,   0,   0, 200]], dtype=int64), '             precision    recall  f1-score   support\n\n      APPLE       1.00      1.00      1.00       466\n    APRICOT       1.00      1.00      1.00       202\n  AUBERGINE       1.00      1.00      1.00       200\n    AVOCADO       1.00      1.00      1.00       209\n     BANANA       1.00      1.00      1.00       213\n       BEAN       1.00      1.00      1.00       195\n      BREAD       1.00      1.00      1.00       220\n        BUN       1.00      1.00      1.00       205\n     CARROT       1.00      1.00      1.00       199\n     CHEESE       1.00      1.00      1.00       194\n   CUCUMBER       1.00      1.00      1.00       196\n      DATES       

In [11]:
mlp_bench.evaluate_model(mlp_bench.transformators['V09_PCA_5components'].transform(X_val),y_val,
                         np.unique(y_val),mlp_bench.best_models_dict['KNeighborsClassifier_V09_PCA_5components'])

(0.6463238827486785,
 0.6400731226694368,
 4.467182043098239,
 array([[55,  2,  0, ...,  0, 10,  1],
        [ 0, 43,  0, ...,  0,  0,  0],
        [ 0,  0, 45, ...,  0,  0,  1],
        ...,
        [ 0,  0,  0, ..., 36,  0,  0],
        [ 3,  0,  0, ...,  0, 45,  0],
        [ 1,  0,  0, ...,  0,  0, 38]], dtype=int64),
 '             precision    recall  f1-score   support\n\n      APPLE       0.50      0.43      0.46       127\n    APRICOT       0.74      0.90      0.81        48\n  AUBERGINE       0.88      0.90      0.89        50\n    AVOCADO       0.81      0.85      0.83        41\n     BANANA       0.35      0.30      0.32        37\n       BEAN       0.74      0.76      0.75        55\n      BREAD       0.65      0.58      0.61        67\n        BUN       0.73      0.49      0.59        45\n     CARROT       0.62      0.55      0.58        51\n     CHEESE       0.71      0.80      0.76        56\n   CUCUMBER       0.67      0.80      0.73        54\n      DATES       0.72  

In [12]:
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold(0.01)
selector.fit(X_train)
Xs_train = selector.transform(X_train)
print(X_train.shape)
print(Xs_train.shape)

(8322, 16680)
(8322, 3142)


In [13]:
np.set_printoptions(threshold=np.inf)
print(np.array(X_train.columns[selector.variances_>0.01]))

['Cluster: 0 cspace: RGB; sortCH1; X' 'Cluster: 0 cspace: RGB; sortCH1; Y'
 'Cluster: 0 cspace: RGB; sortCH2; X' 'Cluster: 0 cspace: RGB; sortCH2; Y'
 'Cluster: 0 cspace: RGB; sortCH3; X' 'Cluster: 0 cspace: RGB; sortCH3; Y'
 'Cluster: 0 cspace: RGB; sortX; X' 'Cluster: 0 cspace: RGB; sortX; Y'
 'Cluster: 0 cspace: RGB; sortY; X' 'Cluster: 0 cspace: RGB; sortY; Y'
 'Cluster: 0 cspace: RGB; sortSize; X'
 'Cluster: 0 cspace: RGB; sortSize; Y'
 'Cluster: 1 cspace: RGB; sortCH1; X' 'Cluster: 1 cspace: RGB; sortCH1; Y'
 'Cluster: 1 cspace: RGB; sortCH2; X' 'Cluster: 1 cspace: RGB; sortCH2; Y'
 'Cluster: 1 cspace: RGB; sortCH3; X' 'Cluster: 1 cspace: RGB; sortCH3; Y'
 'Cluster: 1 cspace: RGB; sortX; X' 'Cluster: 1 cspace: RGB; sortX; Y'
 'Cluster: 1 cspace: RGB; sortY; X' 'Cluster: 1 cspace: RGB; sortY; Y'
 'Cluster: 1 cspace: RGB; sortSize; X'
 'Cluster: 1 cspace: RGB; sortSize; Y'
 'Cluster: 2 cspace: RGB; sortCH1; X' 'Cluster: 2 cspace: RGB; sortCH1; Y'
 'Cluster: 2 cspace: RGB; sortCH2; 

 'Cluster: 4 cspace: YDBDR; sortSize; SIZE']


In [14]:
pipes[0].fit(X_train)

Pipeline(memory=None,
     steps=[('variancethreshold', VarianceThreshold(threshold=0.01)), ('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=0.7, random_state=None,
  svd_solver='full', tol=0.0, whiten=True))])

In [59]:
pipe_names = [#'V0001_PCA_80','V0001_PCA_70','V0001_PCA_5components',
              #'V001_PCA_80',
              'V001_PCA_70',
              'V001_PCA_5components',
              #'V01_PCA_90',
              'V01_PCA_85',
              'V01_PCA_80',
              'V01_PCA_5components',
              #'V09_PCA_95',
              'V09_PCA_90',
              'V09_PCA_80',
              'V09_PCA_5components'
            ]
pipes = [
    #make_pipeline(VarianceThreshold(0.001), StandardScaler(),
    #              PCA(n_components = 0.80, whiten = True, svd_solver = 'full')),
    #make_pipeline(VarianceThreshold(0.001), StandardScaler(),
    #              PCA(n_components = 0.70, whiten = True, svd_solver = 'full')),
    #make_pipeline(VarianceThreshold(0.001), StandardScaler(),
    #              PCA(n_components = 5, whiten = True, svd_solver = 'full')),
    
    #make_pipeline(VarianceThreshold(0.01), StandardScaler(),
    #              PCA(n_components = 0.80, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.01), StandardScaler(),
                  PCA(n_components = 0.70, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.01), StandardScaler(),
                  PCA(n_components = 5, whiten = True, svd_solver = 'full')),
    
    #make_pipeline(VarianceThreshold(0.1), StandardScaler(),
    #              PCA(n_components = 0.90, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.1), StandardScaler(),
                  PCA(n_components = 0.85, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.1), StandardScaler(),
                  PCA(n_components = 0.80, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.1), StandardScaler(),
                  PCA(n_components = 5, whiten = True, svd_solver = 'full')),
    
    #make_pipeline(VarianceThreshold(0.9), StandardScaler(),
    #              PCA(n_components = 0.95, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.9), StandardScaler(),
                  PCA(n_components = 0.90, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.9), StandardScaler(),
                  PCA(n_components = 0.80, whiten = True, svd_solver = 'full')),
    make_pipeline(VarianceThreshold(0.9), StandardScaler(),
                  PCA(n_components = 5, whiten = True, svd_solver = 'full')),
       ]


#pipe_names = ['V09_PCA_5components']
#pipes = [make_pipeline(VarianceThreshold(0.9), StandardScaler(),
#                  PCA(n_components = 5, whiten = True, svd_solver = 'full'))]
model_names_xgb = ['SVC', 'XGBClassifier_tree', 'XGBClassifier_dart']


models_xgb = [SVC(probability=True),
               XGBClassifier()]#,XGBClassifier(booster='dart')]

xgb_tree_dict = {'n_estimators': [50, 100, 200, 250], 'learning_rate': np.arange(0,1,0.01),'gamma': np.arange(0,10,0.1),
                'max_depth': np.arange(5,25,5), 'min_child_weight': [1, 2, 5, 8, 10],
                'max_delta_step': [0, 1, 5], 'subsample': np.arange(0.1,1,0.1),
                 'colsample_bytree': [0.2, 0.4, 0.6, 0.8, 1.0]}
#xgb_dart_dict = {'sample_type': ['uniform','weighted'], 'normalize_type': ['tree','forest'],
#                'rate_drop': np.arange(0,0.9,0.1), 'skip_drop': np.arange(0,0.5,0.1), 'one_drop': [0,1]}
#xgb_dart_dict = xgb_dart_dict.update(xgb_tree_dict)
#xgb_linear_dict = {'lambda': np.arange(0,2,0.1), 'alpha': np.arange(0,2,0.1)}
##{'C': expon(scale=0.2), 'degree': [2,3,4,5,6,7,8], 'kernel': ['rbf', 'poly', 'linear', 'sigmoid'],
##    'coef0': [0.0, 0.5, 0.75, 1.0], 'shrinking': [True, False], 'tol': np.arange(1e-5,0.7,0.00001)},
models_params_xgb = [
    {'C': expon(scale=0.2), 'degree': [2,3,4,5,6,7,8], 'kernel': ['rbf', 'poly', 'linear', 'sigmoid'],
    'coef0': [0.0, 0.5, 0.75, 1.0], 'shrinking': [True, False], 'tol': np.arange(1e-2,0.7,0.01)},
    xgb_tree_dict#, xgb_dart_dict
    
]

In [60]:
xgb_bench = classmod.ModelsBenchmark(model_names_xgb, models_xgb, models_params_xgb)
xgb_bench.fitTransformators(names=pipe_names,
                             transformators=pipes,
                             X=X_train)

In [53]:
XGBClassifier(booster='dart').get_params().keys()

dict_keys(['base_score', 'booster', 'colsample_bylevel', 'colsample_bytree', 'gamma', 'learning_rate', 'max_delta_step', 'max_depth', 'min_child_weight', 'missing', 'n_estimators', 'n_jobs', 'nthread', 'objective', 'random_state', 'reg_alpha', 'reg_lambda', 'scale_pos_weight', 'seed', 'silent', 'subsample'])

In [61]:
y_labels = np.unique(y_train)
score_dict = {'Accuracy': make_scorer(accuracy_score), 'F1': make_scorer(f1_score, average='weighted'),
             'Log_Loss': make_scorer(log_loss,normalize=True,greater_is_better=False, needs_proba=True)} 
xgb_bench.runBenchmarkOnLoaded(X_train, y_train,  prio_score='Log_Loss', folds=3, iters=5, randomized=True,
                                    verbosity=3, print_shapes=True,
                               scoring=score_dict)
rootDir = '''D:/PROGRAMMING_PROJS/FoodMachineLearning/FoodMLrepo/FoodML/foodml_src/'''
xgb_bench.saveToPickles(rootDir,'randomized_results_svc_xgboost')

Before PCA, x: 8322; y: 16680
After PCA, x: 8322; y: 89
Finding best parameters for SVC_V001_PCA_70:
Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  3.7min finished


Evaluating SVC_V001_PCA_70...

Finding best parameters for XGBClassifier_tree_V001_PCA_70:
Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  6.2min finished


Evaluating XGBClassifier_tree_V001_PCA_70...



...

C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Finding best parameters for XGBClassifier_tree_V09_PCA_5components:
Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.6min finished


Evaluating XGBClassifier_tree_V09_PCA_5components...



C:\Users\AdamT\Anaconda3\envs\foodmlenv\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [62]:
xgb_bench.best_models_results

{'SVC_V001_PCA_70': (0.9858207161739966,
  0.9858756077124561,
  0.18143819205810097,
  array([[458,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   3,   0,   0,   0,   3,   1,   0,   0,   1,   0],
         [  0, 201,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0, 198,   0,   0,   0,   0,   0,   1,   0,   0,   1,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0, 209,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0, 